### JsonOutputParser

    - 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 parser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [5]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtage: str = Field(description="해시태그 형식의 키워드(2개 이상)")

# 파서 설정
output_parser = JsonOutputParser(pydantic_object=Topic)

In [6]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtage": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtage", "type": "string"}}, "required": ["description", "hashtage"]}
```


In [7]:
from langchain_core.prompts import ChatPromptTemplate

question = "점심 먹고 졸린 이유를 알려주세요"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format : {format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

In [8]:
chain = prompt | model | output_parser

answer = chain.invoke({"question": question})

In [9]:
type(answer)

dict

In [ ]:
print(answer)

{'description': '점심 식사 후 졸림은 주로 혈당 수치의 변화와 소화 과정에서의 에너지 소비로 인해 발생합니다. 식사 후 혈당이 급격히 상승하면 인슐린 분비가 증가하고, 이는 혈당을 낮추기 위해 작용합니다. 이 과정에서 뇌로 가는 혈류가 줄어들어 졸음을 유발할 수 있습니다. 또한, 소화 과정에서 많은 에너지가 사용되기 때문에 몸이 피로를 느낄 수 있습니다.', 'hashtage': '#점심졸림 #혈당변화 #소화과정'}


### Pydantic을 사용하지 않고 JsonOutputParser 사용

In [11]:
output_parser = JsonOutputParser()

question = "잠에서 깨는 방법에 대해 알려주세요. 방법에 대한 설명은 `descrikption`에 관련키워드는 `hashtags`에 담아주세요"

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format: {format_instructions}\nQuestion:\n{question}")
    ]
)

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | model | output_parser

response = chain.invoke({"question" : question})

print(response)

{'methods': [{'description': '알람 시계를 사용하여 규칙적인 시간에 일어나도록 합니다.', 'hashtags': ['#알람', '#규칙적인시간', '#일어남']}, {'description': '햇빛을 받아 자연스럽게 잠에서 깨어납니다.', 'hashtags': ['#햇빛', '#자연스러운기상', '#일출']}, {'description': '스트레칭이나 가벼운 운동으로 몸을 깨웁니다.', 'hashtags': ['#스트레칭', '#가벼운운동', '#기상운동']}, {'description': '커피나 차와 같은 카페인 음료를 마십니다.', 'hashtags': ['#커피', '#차', '#카페인']}, {'description': '시원한 물로 세수를 하여 잠을 깹니다.', 'hashtags': ['#세수', '#시원한물', '#기상']}]}
